# In this notebook, we will use our pre-trained model to predict on test set.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['wedding-images', 'share-img']


## 1. Load pre-trained model

In [2]:
import os
import tensorflow as tf
import h5py

import os.path
from pathlib import Path

dir_pretrained_models = '../input/wedding-images'
dir_dataset = '../input/share-img'
dir_dataset_wedding_images= os.path.join(dir_dataset, 'test_set')
#dir_dataset_wedding_images = Path(dir_dataset_wedding_images).as_posix()
dir_testset = os.path.join(dir_dataset, 'test_set/img1')
#dir_dataset_img1 = Path(dir_dataset_img1).as_posix()

pretrained_models = os.listdir(dir_pretrained_models)
pretrained_models = [ i for i in pretrained_models if i.endswith('h5')]
print (pretrained_models)

if pretrained_models:
    new_model = tf.keras.models.load_model(os.path.join(dir_pretrained_models, pretrained_models[0]))
    new_model.summary()

['model_multi_label.h5']
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
batch_normalization_v1 (Batc (None, 1024)              4096      
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
______________

## 2. Load test dataset and make prediction

In [3]:
import pandas as pd
from glob import glob

#pretrained_models = os.listdir(dir_testset)
#print (pretrained_models)

tmp_path = os.path.join(dir_testset,'*/*.jpg')
test_df = pd.DataFrame({'path': glob(tmp_path)})
#test_df.path.map(lambda x: Path(x).as_posix())

#df['id'] = df.path.map(lambda x: x.split('/')[3].split("\\")[1]) # on Windows

test_df.head(10)

#test_df = test_df.sample(100, random_state = 101)

,path
0,../input/share-img/test_set/img1/123459078/d28...
1,../input/share-img/test_set/img1/123459078/b61...
2,../input/share-img/test_set/img1/123459078/1dc...
3,../input/share-img/test_set/img1/123459078/ba6...
4,../input/share-img/test_set/img1/123459078/a9d...
5,../input/share-img/test_set/img1/123459078/46e...
6,../input/share-img/test_set/img1/123459078/6ad...
7,../input/share-img/test_set/img1/123459078/038...
8,../input/share-img/test_set/img1/123459078/5a9...
9,../input/share-img/test_set/img1/123459061/752...


In [4]:
test_df.shape

(2108, 1)

In [5]:
import numpy as np
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img


train_batch_size = 32
val_batch_size = 32
num_test_samples = len(test_df)
test_batch_size = 1

IMAGE_SIZE = 224

#train_steps = np.ceil(num_train_samples / train_batch_size)
test_steps= np.ceil(num_test_samples / test_batch_size)
       
testgen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x)

# Note: shuffle=False causes the test dataset to not be shuffled
test_gen = testgen.flow_from_dataframe(dataframe=test_df,
                                        directory=None,
                                        x_col="path",
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=test_batch_size,
                                        class_mode=None,
                                        shuffle=False)

Found 2108 images.


Using TensorFlow backend.


In [6]:
test_gen.reset()
pred=new_model.predict_generator(test_gen,steps=test_steps,verbose=1)

2108/2108 [==============================] - 46s 22ms/step


In [7]:
#pred_bool = (pred >0.5)


In [8]:
#predictions = pred_bool.astype(int)
#columns should be the same order of y_col
cols = ['zhongguofeng', 'zhongshi', 'zhuti', 'weimei', 'fugu', 'daqi', 'xiaoqingxin', 'menghuan', 'senxi', 'jianjie', 'xishi', 'hanshi']
results=pd.DataFrame(pred, columns=cols)
results["Filenames"]=test_gen.filenames
ordered_cols=["Filenames"]+cols
results=results[ordered_cols]#To get the same column order
#results.to_csv("results.csv",index=False)
results_bk = results
results.head(20)



,Filenames,zhongguofeng,zhongshi,zhuti,weimei,fugu,daqi,xiaoqingxin,menghuan,senxi,jianjie,xishi,hanshi
0,../input/share-img/test_set/img1/123459078/d28...,0.000083,0.001153,0.019763,0.003025,0.919168,0.334485,0.000311,0.002121,0.974706,0.736684,0.461187,0.000109
1,../input/share-img/test_set/img1/123459078/b61...,0.000024,0.000101,0.002309,0.000769,0.206885,0.139868,0.765661,0.006325,0.484193,0.006717,0.995060,0.000417
2,../input/share-img/test_set/img1/123459078/1dc...,0.000657,0.000243,0.132608,0.004951,0.992660,0.380109,0.006603,0.007523,0.063161,0.153317,0.637790,0.001600
3,../input/share-img/test_set/img1/123459078/ba6...,0.000946,0.001051,0.899695,0.003607,0.989443,0.002154,0.049935,0.013344,0.839024,0.003489,0.293455,0.000324
4,../input/share-img/test_set/img1/123459078/a9d...,0.003645,0.007147,0.240246,0.006902,0.218343,0.860659,0.534163,0.022132,0.042107,0.014019,0.253762,0.000833
5,../input/share-img/test_set/img1/123459078/46e...,0.001263,0.001344,0.012736,0.096137,0.016811,0.009975,0.878873,0.010430,0.966047,0.075828,0.195217,0.000894
6,../input/share-img/test_set/img1/123459078/6ad...,0.000341,0.000222,0.567696,0.002033,0.244009,0.010494,0.405210,0.004746,0.259008,0.008642,0.981942,0.001204
7,../input/share-img/test_set/img1/123459078/038...,0.001190,0.002074,0.594914,0.005644,0.828835,0.002874,0.906209,0.001374,0.663433,0.052326,0.083573,0.002346
8,../input/share-img/test_set/img1/123459078/5a9...,0.002989,0.007816,0.005820,0.003680,0.985027,0.023127,0.000963,0.013291,0.709623,0.020249,0.921703,0.000083
9,../input/share-img/test_set/img1/123459061/752...,0.960864,0.315267,0.131841,0.865059,0.002223,0.154314,0.027744,0.028843,0.001255,0.008192,0.147380,0.003947


## 3. Post-processing
1. In current system, supported number of labels is 1 ~ 3.
2. labels is marked for cases, so all the images for same case should have same labels.

In [9]:
results = results.drop(['Filenames'], axis=1)

results.T.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107
zhongguofeng,0.000083,0.000024,0.000657,0.000946,0.003645,0.001263,0.000341,0.001190,0.002989,0.960864,0.017688,0.595653,0.001617,0.022866,0.000566,0.006313,0.655378,0.000539,0.032688,0.001588,0.002528,0.014556,0.094242,0.000440,0.099038,0.003235,0.009967,0.626033,0.001297,0.009735,0.001077,0.106250,0.013108,0.005769,0.000256,0.002100,0.006361,0.006703,0.003144,0.000181,...,0.082466,0.006846,0.000142,0.034803,0.043412,0.000130,0.003263,0.000451,0.005068,0.618919,0.596555,0.883541,0.234216,0.001602,0.977037,0.091236,0.558656,0.007247,0.995808,0.905439,0.107598,0.638792,0.000878,0.000559,0.037325,0.000451,0.000248,0.000250,0.000500,0.000108,0.000069,0.003051,0.000974,0.000593,0.000398,0.000455,0.000434,0.001265,0.002821,0.000073
zhongshi,0.001153,0.000101,0.000243,0.001051,0.007147,0.001344,0.000222,0.002074,0.007816,0.315267,0.013062,0.041380,0.001546,0.003710,0.000670,0.000750,0.113648,0.000664,0.008269,0.002109,0.000254,0.008640,0.013073,0.000322,0.023241,0.003727,0.004589,0.075799,0.000805,0.005318,0.001867,0.012381,0.005125,0.005990,0.000334,0.003240,0.030560,0.057796,0.012786,0.000281,...,0.254102,0.011383,0.000312,0.106327,0.084190,0.001294,0.003400,0.002017,0.081554,0.737607,0.683676,0.972043,0.455895,0.001997,0.954385,0.704653,0.549700,0.023779,0.921383,0.832085,0.224464,0.718194,0.000358,0.000274,0.004992,0.000187,0.000526,0.000502,0.001172,0.000061,0.000075,0.000688,0.000101,0.000570,0.000655,0.000580,0.001323,0.002783,0.001413,0.000109
zhuti,0.019763,0.002309,0.132608,0.899695,0.240246,0.012736,0.567696,0.594914,0.005820,0.131841,0.233414,0.512866,0.512029,0.517441,0.842754,0.578051,0.147177,0.002086,0.074580,0.591858,0.490215,0.220883,0.480576,0.762012,0.177867,0.032787,0.104323,0.038048,0.013084,0.005782,0.023167,0.022986,0.080881,0.949937,0.020957,0.019899,0.118567,0.017628,0.006904,0.001478,...,0.195764,0.728501,0.102806,0.744426,0.937434,0.000665,0.009106,0.001609,0.033035,0.093377,0.005534,0.024146,0.296817,0.671816,0.058934,0.081102,0.111488,0.016226,0.244149,0.391256,0.173295,0.049575,0.087229,0.523562,0.823539,0.246783,0.506273,0.017787,0.072347,0.331223,0.095095,0.024238,0.994770,0.944898,0.074608,0.030796,0.008656,0.100585,0.354631,0.233571
weimei,0.003025,0.000769,0.004951,0.003607,0.006902,0.096137,0.002033,0.005644,0.003680,0.865059,0.021912,0.865167,0.017244,0.199968,0.042629,0.127516,0.028107,0.037221,0.009526,0.002480,0.112792,0.047712,0.025501,0.000866,0.012199,0.001714,0.885721,0.735582,0.807829,0.817089,0.991478,0.573315,0.998685,0.262170,0.935664,0.260863,0.036507,0.441807,0.967645,0.044623,...,0.006615,0.007610,0.032771,0.000922,0.001890,0.195923,0.122787,0.117484,0.003658,0.017414,0.054887,0.001899,0.002702,0.004067,0.039097,0.000813,0.027254,0.000329,0.005108,0.003866,0.016535,0.021586,0.866194,0.943531,0.143824,0.990500,0.987869,0.996913,0.262731,0.997441,0.992262,0.020300,0.794693,0.329872,0.795328,0.787668,0.989446,0.835067,0.995664,0.977293
fugu,0.919168,0.206885,0.992660,0.989443,0.218343,0.016811,0.244009,0.828835,0.985027,0.002223,0.070754,0.000651,0.325388,0.948564,0.029257,0.036724,0.011216,0.002690,0.924718,0.044796,0.979033,0.538684,0.021463,0.002280,0.001824,0.597735,0.000537,0.001094,0.003849,0.000454,0.000531,0.000374,0.021888,0.000166,0.006134,0.000627,0.004508,0.077529,0.000204,0.001269,...,0.230423,0.974995,0.005061,0.975193,0.923080,0.042675,0.821052,0.799205,0.361703,0.501707,0.764853,0.223754,0.655854,0.436670,0.080942,0.947421,0.539198,0.995718,0.005968,0.930848,0.551745,0.227178,0.007121,0.000294,0.032453,0.001545,0.002422,0.000365,0.023912,0.001252,0.000764,0.037740,0.000385,0.001722,0.002983,0.008196,0.005578,0.016758,0.000937,0.000656
daqi,0.334485,0.139868,0.

In [10]:
results_final = pd.DataFrame()

for i in range(num_test_samples):
    
    results_top3 = results.T.iloc[:,i].apply(lambda x: x if x in list(results.T.iloc[:,i].nlargest(3)) else 0)
    results_final = pd.concat([results_final, results_top3], axis=1)
    #print (results_final)

#results_final = results_final.T
results_final.head(20)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107
daqi,0.000000,0.000000,0.380109,0.000000,0.860659,0.000000,0.000000,0.000000,0.000000,0.000000,0.467629,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.739556,0.199014,0.512155,0.377723,0.000000,0.000000,0.994983,0.973978,0.990245,0.965351,0.996155,...,0.681544,0.000000,0.819695,0.000000,0.000000,0.989556,0.612376,0.510855,0.990787,0.000000,0.000000,0.459345,0.633556,0.619254,0.151395,0.321016,0.594409,0.000000,0.000000,0.000000,0.399919,0.627105,0.471546,0.931248,0.000000,0.917697,0.000000,0.959422,0.947894,0.998066,0.998052,0.000000,0.884526,0.776975,0.905028,0.997851,0.964170,0.970462,0.000000,0.990604
fugu,0.919168,0.000000,0.992660,0.989443,0.000000,0.000000,0.000000,0.828835,0.985027,0.000000,0.000000,0.000000,0.325388,0.948564,0.000000,0.000000,0.000000,0.000000,0.924718,0.000000,0.979033,0.538684,0.000000,0.000000,0.000000,0.597735,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.974995,0.000000,0.975193,0.923080,0.000000,0.821052,0.799205,0.361703,0.501707,0.764853,0.000000,0.655854,0.436670,0.000000,0.947421,0.000000,0.995718,0.000000,0.930848,0.551745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
hanshi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.960635,0.000000,0.572745,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
jianjie,0.736684,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.893377,0.000000,0.000000,0.000000,0.000000,0.731334,0.000000,0.112846,0.000000,0.555612,0.000000,0.000000,0.000000,0.000000,0.543705,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.580925,0.957272,0.747398,0.000000,0.000000,0.664249,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
menghuan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.694683,0.883999,0.843461,0.969662,0.989063,0.000000,0.000000,0.815157,0.000000,0.000000,0.000000,0.000000,0.605313,0.889108,...,0.000000,0.000000,0.585586,0.000000,0.000000,0.656917,0.520574,0.751629,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.696303,0.997439,0.000000,0.899101,0.995408,0.995155,0.982873,0.837564,0.994565,0.122941,0.000000,0.887664,0.995249,0.996439,0.997254,0.931841,0.315152,0.999218
senxi,0.974706,0.484193,0.00000

In [11]:
#results_final = results_final.T
#results_final.head(20)

In [12]:
def keep_atleast1_below_half(x):
    ret = []
    if x.max() <= 0.5:
        for i in x:
            if i == x.max():
                ret.append(1)
            else:
                ret.append(0)
    else:
        ret = x
        
    return ret
#lambda x: 0 if (x.max()<=0.5 and x<x.max()) else x
results_final = results_final.apply(keep_atleast1_below_half)
results_final.head(12)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107
daqi,0.000000,0.000000,0.380109,0.000000,0.860659,0.000000,0.000000,0.000000,0.000000,0.000000,0.467629,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.739556,0.199014,0.512155,0.377723,0.000000,0.000000,0.994983,0.973978,0.990245,0.965351,0.996155,...,0.681544,0.000000,0.819695,0.000000,0.000000,0.989556,0.612376,0.510855,0.990787,0.000000,0.000000,0.459345,0.633556,0.619254,0.151395,0.321016,0.594409,0.000000,0.000000,0.000000,0.399919,0.627105,0.471546,0.931248,0.000000,0.917697,0.000000,0.959422,0.947894,0.998066,0.998052,0.000000,0.884526,0.776975,0.905028,0.997851,0.964170,0.970462,0.000000,0.990604
fugu,0.919168,0.000000,0.992660,0.989443,0.000000,0.000000,0.000000,0.828835,0.985027,0.000000,0.000000,0.000000,0.325388,0.948564,0.000000,0.000000,0.000000,0.000000,0.924718,0.000000,0.979033,0.538684,0,0.000000,0.000000,0.597735,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.974995,0.000000,0.975193,0.923080,0.000000,0.821052,0.799205,0.361703,0.501707,0.764853,0.000000,0.655854,0.436670,0.000000,0.947421,0.000000,0.995718,0.000000,0.930848,0.551745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
hanshi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.960635,0.000000,0.572745,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
jianjie,0.736684,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.893377,0.000000,0.000000,0.000000,0.000000,0.731334,0.000000,0.112846,0.000000,0.555612,0.000000,0.000000,0,0.000000,0.543705,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.580925,0.957272,0.747398,0.000000,0.000000,0.664249,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
menghuan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350691,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.694683,0.883999,0.843461,0.969662,0.989063,0.000000,0.000000,0.815157,0.000000,0.000000,0.000000,0.000000,0.605313,0.889108,...,0.000000,0.000000,0.585586,0.000000,0.000000,0.656917,0.520574,0.751629,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.696303,0.997439,0.000000,0.899101,0.995408,0.995155,0.982873,0.837564,0.994565,0.122941,0.000000,0.887664,0.995249,0.996439,0.997254,0.931841,0.315152,0.999218
senxi,0.974706,0.484193,0.000000,0.839024,0.000000,0.966047,0.0000

In [13]:
def probability_to_predict(x):
    ret = []
    num_unique = len(x.unique())
    if num_unique == 2: # only one max number
        for i in x:
            if i !=0:
                ret.append(1)
            else:
                ret.append(0)
    elif num_unique>2:
        for i in x:
            if i > 0.5:
                ret.append(1)
            else:
                ret.append(0)
    return ret
#lambda x: 0 if (x.max()<=0.5 and x<x.max()) else x
results_final = results_final.apply(probability_to_predict)
results_final.head(12)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107
daqi,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1,1,...,1,0,1,0,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0,1,0,1,1,1,1,0,1,1,1,1,1,1,0,1
fugu,1,0,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,0,1,1,0,1,1,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
hanshi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
jianjie,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
menghuan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,0,0,0,0,1,1,...,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1
senxi,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
weimei,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,1,1,0,1,0,1,1,1,1,1,1
xiaoqingxin,0,1,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
xishi,0,1,1,0,0,0,1,0,1,0,0,0,0,1,1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
zhongguofeng,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# trasporse back
results_final = results_final.T
results_final.head(10)

,daqi,fugu,hanshi,jianjie,menghuan,senxi,weimei,xiaoqingxin,xishi,zhongguofeng,zhongshi,zhuti
0,0,1,0,1,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,0,0
2,0,1,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,1,0,0,0,0,0,1
4,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,1
7,0,1,0,0,0,1,0,1,0,0,0,0
8,0,1,0,0,0,1,0,0,1,0,0,0
9,0,0,0,0,0,0,1,0,0,1,0,0


In [15]:

results_final = pd.concat([results_bk['Filenames'], results_final], axis=1)

results_final.head(10)

,Filenames,daqi,fugu,hanshi,jianjie,menghuan,senxi,weimei,xiaoqingxin,xishi,zhongguofeng,zhongshi,zhuti
0,../input/share-img/test_set/img1/123459078/d28...,0,1,0,1,0,1,0,0,0,0,0,0
1,../input/share-img/test_set/img1/123459078/b61...,0,0,0,0,0,0,0,1,1,0,0,0
2,../input/share-img/test_set/img1/123459078/1dc...,0,1,0,0,0,0,0,0,1,0,0,0
3,../input/share-img/test_set/img1/123459078/ba6...,0,1,0,0,0,1,0,0,0,0,0,1
4,../input/share-img/test_set/img1/123459078/a9d...,1,0,0,0,0,0,0,1,0,0,0,0
5,../input/share-img/test_set/img1/123459078/46e...,0,0,0,0,0,1,0,1,0,0,0,0
6,../input/share-img/test_set/img1/123459078/6ad...,0,0,0,0,0,0,0,0,1,0,0,1
7,../input/share-img/test_set/img1/123459078/038...,0,1,0,0,0,1,0,1,0,0,0,0
8,../input/share-img/test_set/img1/123459078/5a9...,0,1,0,0,0,1,0,0,1,0,0,0
9,../input/share-img/test_set/img1/123459061/752...,0,0,0,0,0,0,1,0,0,1,0,0


In [16]:
results_final.to_csv("results.csv",index=False)
